
# 1. IMPORT

In [54]:
import pandas as pd
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import MinMaxScaler
import scipy.cluster.hierarchy as shc
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split 
import matplotlib.patheffects as PathEffects
import seaborn as sns
import numpy as np
# importing required libraries for normalizing data
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import LabelBinarizer

In [55]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from keras.layers import Dense, LSTM, MaxPool1D, Flatten, Dropout,Conv1D # importing dense layer
from keras.models import Sequential #importing Sequential layer
from keras.layers import Input
from keras.models import Model
# representation of model layers
from keras.utils.vis_utils import plot_model

In [56]:
feature=["duration","protocol_type","service","flag","src_bytes","dst_bytes","land","wrong_fragment","urgent","hot",
          "num_failed_logins","logged_in","num_compromised","root_shell","su_attempted","num_root","num_file_creations","num_shells",
          "num_access_files","num_outbound_cmds","is_host_login","is_guest_login","count","srv_count","serror_rate","srv_serror_rate",
          "rerror_rate","srv_rerror_rate","same_srv_rate","diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count", 
          "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate","dst_host_srv_diff_host_rate","dst_host_serror_rate",
          "dst_host_srv_serror_rate","dst_host_rerror_rate","dst_host_srv_rerror_rate","label","difficulty"]

In [57]:
train='./KDDTrain+.txt'
test='./KDDTest+.txt'
data=pd.read_csv(train,names=feature)
test_data=pd.read_csv(test,names=feature)
df= pd.concat([data, test_data], ignore_index=True)

In [58]:
# phân loại lớp 
def change_label(df):
  df.label.replace(['apache2','back','land','neptune','mailbomb','pod','processtable','smurf','teardrop','udpstorm','worm'],'Dos',inplace=True)
  df.label.replace(['ftp_write','guess_passwd','httptunnel','imap','multihop','named','phf','sendmail','snmpgetattack','snmpguess','spy','warezclient','warezmaster','xlock','xsnoop'],'R2L',inplace=True)      
  df.label.replace(['ipsweep','mscan','nmap','portsweep','saint','satan'],'Probe',inplace=True)
  df.label.replace(['buffer_overflow','loadmodule','perl','ps','rootkit','sqlattack','xterm'],'U2R',inplace=True)

change_label(df)
# change_label(test_data)

In [59]:
# xóa 'difficulty_level'
df.drop(['difficulty'],axis=1,inplace=True)
#test_data.drop(['difficulty'],axis=1,inplace=True)


# 2. BIẾN ĐỔI VÀ CHUẨN HÓA

In [60]:
# one-hot-encoding categorical columns
df = pd.get_dummies(df,columns=['protocol_type','service','flag'],prefix="",prefix_sep="")  
#print(data.shape)
#test_data = pd.get_dummies(test_data,columns=['protocol_type','service','flag'],prefix="",prefix_sep="")  

In [61]:
# tạo datafreame labels (Dos,Probe,R2L,U2R,normal)
label = pd.DataFrame(df.label)
#label_test = pd.DataFrame(test_data.label)

In [62]:
minmax_scaler = MinMaxScaler()
def normalization(df, col):
    for i in col:
        arr = df[i]
        arr = np.array(arr)
        df[i] = minmax_scaler.fit_transform(arr.reshape(len(arr), 1))
    return df
#chuẩn hóa dữ liệu số của data
numeric_col = df.select_dtypes(include='number').columns
df = normalization(df, numeric_col)

In [63]:
le2 = preprocessing.LabelEncoder()
enc_label = label.apply(le2.fit_transform)
df['intrusion'] = enc_label
print(df.shape)
df

(148517, 124)


C:\Users\satra\AppData\Local\Temp\ipykernel_11144\2616363727.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['intrusion'] = enc_label


,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,RSTO,RSTOS0,RSTR,S0,S1,S2,S3,SF,SH,intrusion
0,0.0,3.558064e-07,0.000000e+00,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4
1,0.0,1.057999e-07,0.000000e+00,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4
2,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0
3,0.0,1.681203e-07,6.223962e-06,0.0,0.0,0.0,0.000000,0.0,1.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4
4,0.0,1.442067e-07,3.206260e-07,0.0,0.0,0.0,0.000000,0.0,1.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148512,0.0,5.753774e-07,2.542106e-07,0.0,0.0,0.0,0.000000,0.0,1.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4
148513,0.0,2.297162e-07,7.160648e-07,0.0,0.0,0.0,0.000000,0.0,1.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4
148514,0.0,3.952277e-05,6.346868e-06,0.0,0.0,0.0,0.019802,0.0,1.0,0.000134,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0
148515,0.0,3.043558e-08,3.206260e-08,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4


In [64]:
# Phân 4 loại tấn công của bộ train vào dataframe
# Tạo 4 dataframe mới
DoS_df = pd.DataFrame()
Probe_df = pd.DataFrame()
R2L_df = pd.DataFrame()
U2R_df = pd.DataFrame()
normal_df = pd.DataFrame()

# Nhóm các mẫu theo nhãn và lưu vào các dataframe tương ứng
for label, group in df.groupby('label'):
    if label == 'Dos':
        DoS_df = pd.concat([DoS_df, group], axis=0)
    elif label == 'Probe' :
        Probe_df = pd.concat([Probe_df, group], axis=0)
    elif label == 'R2L' :
        R2L_df = pd.concat([R2L_df, group], axis=0)
    elif label == 'U2R' :
        U2R_df = pd.concat([U2R_df, group], axis=0)
    elif label == 'normal':
        normal_df = pd.concat([normal_df, group], axis=0)


In [66]:
DoS_df.drop(labels= ['label'], axis=1, inplace=True)

In [67]:
YDoS_train = DoS_df['intrusion']
XDoS_train = DoS_df.drop(labels= ['intrusion'], axis=1)

In [68]:
from sklearn.preprocessing import LabelBinarizer
YDoS_train = LabelBinarizer().fit_transform(YDoS_train)

XDoS_train=np.array(XDoS_train)
YDoS_train=np.array(YDoS_train)

In [69]:
XDoS_train, XDoS_test, YDoS_train, YDoS_test = train_test_split(XDoS_train, YDoS_train, test_size=0.2, random_state=42)

In [70]:
# reshape input to be [samples, time steps, features]
XDoS_train = np.reshape(XDoS_train, ( XDoS_train.shape[0], XDoS_train.shape[1],1 ))
XDoS_test = np.reshape(XDoS_test, ( XDoS_test.shape[0],  XDoS_test.shape[1],1 ))

In [71]:
print(XDoS_train.shape)
print(XDoS_test.shape)
print(YDoS_train.shape)
print(YDoS_test.shape)

(104352, 122, 1)
(26089, 122, 1)
(104352, 1)
(26089, 1)


In [72]:
XDoS_train.shape

(104352, 122, 1)

# K-MEANS

In [ ]:
n_clusters = 8
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
kmeans.fit(X_train)

# Chiết xuất đặc trưng của các cụm tấn công
cluster_features = []
for i in range(n_clusters):
    indices = np.where(kmeans.labels_ == i)[0]
    cluster_data = X_train[indices]
    cluster_feature = np.mean(cluster_data, axis=0)
    cluster_features.append(cluster_feature)

# Sử dụng đặc trưng để huấn luyện mô hình
clf = RandomForestClassifier(random_state=42)
clf.fit(cluster_features, range(n_clusters))

# Dự đoán cụm của các dữ liệu test
test_cluster_labels = kmeans.predict(X_test)

# Dự đoán nhãn tấn công của các dữ liệu test
test_pred = np.zeros_like(y_test)
for i in range(len(test_pred)):
    cluster_feature = cluster_features[test_cluster_labels[i]]
    test_pred[i] = clf.predict([cluster_feature])
    
# Đánh giá kết quả bằng confusion matrix
confusion = confusion_matrix(y_test, test_pred)
print(confusion)

# MÔ HÌNH MARKOV

# MÔ HÌNH CNN

#  - DoS

In [73]:
# create CNN model
model1 = Sequential()

# add convolutional layer with 32 filters, kernel size of 3x3 and activation function ReLU
model1.add(Conv1D(32, kernel_size=3,activation='relu', input_shape=( XDoS_train.shape[1],XDoS_train.shape[2])))

# add max pooling layer
model1.add(MaxPool1D(pool_size=3))

# add flatten layer to convert output of convolutional layer to 1D array
model1.add(Flatten())


# add dense layer with 128 neurons and activation function ReLU
model1.add(Dense(128, activation='relu'))

# add dropout layer with rate of 0.2 to prevent overfitting
model1.add(Dropout(0.2))

# add output layer with 1 neuron and sigmoid activation function for binary classification
model1.add(Dense(1, activation='sigmoid'))

# compile model with binary crossentropy loss function and Adam optimizer
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# print model summary
model1.summary()


Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_6 (Conv1D)           (None, 120, 32)           128       
                                                                 
 max_pooling1d_5 (MaxPooling  (None, 40, 32)           0         
 1D)                                                             
                                                                 
 flatten_5 (Flatten)         (None, 1280)              0         
                                                                 
 dense_10 (Dense)            (None, 128)               163968    
                                                                 
 dropout_13 (Dropout)        (None, 128)               0         
                                                                 
 dense_11 (Dense)            (None, 1)                 129       
                                                     

In [74]:
history = model1.fit(XDoS_train, YDoS_train, batch_size=32, epochs=50, validation_data=(XDoS_test, YDoS_test))

Epoch 1/50
3261/3261 [==============================] - 19s 5ms/step - loss: 0.0554 - accuracy: 0.9841 - val_loss: 0.0321 - val_accuracy: 0.9910
Epoch 2/50
3261/3261 [==============================] - 18s 6ms/step - loss: 0.0270 - accuracy: 0.9911 - val_loss: 0.0201 - val_accuracy: 0.9934
Epoch 3/50
3261/3261 [==============================] - 17s 5ms/step - loss: 0.0161 - accuracy: 0.9945 - val_loss: 0.0109 - val_accuracy: 0.9972
Epoch 4/50
3261/3261 [==============================] - 19s 6ms/step - loss: 0.0122 - accuracy: 0.9958 - val_loss: 0.0107 - val_accuracy: 0.9971
Epoch 5/50
3261/3261 [==============================] - 17s 5ms/step - loss: 0.0115 - accuracy: 0.9962 - val_loss: 0.0120 - val_accuracy: 0.9965
Epoch 6/50
3261/3261 [==============================] - 18s 6ms/step - loss: 0.0099 - accuracy: 0.9967 - val_loss: 0.0126 - val_accuracy: 0.9969
Epoch 7/50
3261/3261 [==============================] - 19s 6ms/step - loss: 0.0089 - accuracy: 0.9970 - val_loss: 0.0106 - val_ac

#    - Probe

In [75]:
Probe_df.drop(labels= ['label'], axis=1, inplace=True)
YProbe_train = Probe_df['intrusion']
XProbe_train = Probe_df.drop(labels= ['intrusion'], axis=1)
from sklearn.preprocessing import LabelBinarizer
YProbe_train = LabelBinarizer().fit_transform(YProbe_train)

XProbe_train=np.array(XProbe_train)
YProbe_train=np.array(YProbe_train)
XProbe_train, XProbe_test, YProbe_train, YProbe_test = train_test_split(XProbe_train, YProbe_train, test_size=0.2, random_state=42)

# reshape input to be [samples, time steps, features]
XProbe_train = np.reshape(XProbe_train, ( XProbe_train.shape[0], XProbe_train.shape[1],1 ))
XProbe_test = np.reshape(XProbe_test, ( XProbe_test.shape[0],  XProbe_test.shape[1],1 ))

In [76]:
# create CNN model
model2 = Sequential()

# add convolutional layer with 32 filters, kernel size of 3x3 and activation function ReLU
model2.add(Conv1D(32, kernel_size=3,activation='relu', input_shape=( XProbe_train.shape[1],XProbe_train.shape[2])))

# add max pooling layer
model2.add(MaxPool1D(pool_size=3))

# add flatten layer to convert output of convolutional layer to 1D array
model2.add(Flatten())


# add dense layer with 128 neurons and activation function ReLU
model2.add(Dense(128, activation='relu'))

# add dropout layer with rate of 0.2 to prevent overfitting
model2.add(Dropout(0.2))

# add output layer with 1 neuron and sigmoid activation function for binary classification
model2.add(Dense(1, activation='sigmoid'))

# compile model with binary crossentropy loss function and Adam optimizer
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# print model summary
model2.summary()


Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_7 (Conv1D)           (None, 120, 32)           128       
                                                                 
 max_pooling1d_6 (MaxPooling  (None, 40, 32)           0         
 1D)                                                             
                                                                 
 flatten_6 (Flatten)         (None, 1280)              0         
                                                                 
 dense_12 (Dense)            (None, 128)               163968    
                                                                 
 dropout_14 (Dropout)        (None, 128)               0         
                                                                 
 dense_13 (Dense)            (None, 1)                 129       
                                                     

In [77]:
history = model2.fit(XProbe_train, YProbe_train, batch_size=32, epochs=50, validation_data=(XProbe_test, YProbe_test))

Epoch 1/50
2279/2279 [==============================] - 16s 6ms/step - loss: 0.0483 - accuracy: 0.9837 - val_loss: 0.0305 - val_accuracy: 0.9900
Epoch 2/50
2279/2279 [==============================] - 15s 6ms/step - loss: 0.0301 - accuracy: 0.9908 - val_loss: 0.0234 - val_accuracy: 0.9919
Epoch 3/50
2279/2279 [==============================] - 19s 8ms/step - loss: 0.0250 - accuracy: 0.9923 - val_loss: 0.0199 - val_accuracy: 0.9945
Epoch 4/50
2279/2279 [==============================] - 21s 9ms/step - loss: 0.0219 - accuracy: 0.9932 - val_loss: 0.0174 - val_accuracy: 0.9954
Epoch 5/50
2279/2279 [==============================] - 37s 16ms/step - loss: 0.0197 - accuracy: 0.9939 - val_loss: 0.0180 - val_accuracy: 0.9949
Epoch 6/50
2279/2279 [==============================] - 17s 8ms/step - loss: 0.0186 - accuracy: 0.9942 - val_loss: 0.0150 - val_accuracy: 0.9951
Epoch 7/50
2279/2279 [==============================] - 16s 7ms/step - loss: 0.0168 - accuracy: 0.9945 - val_loss: 0.0245 - val_a

# - R2L

In [78]:
R2L_df.drop(labels= ['label'], axis=1, inplace=True)
YR2L_train = R2L_df['intrusion']
XR2L_train = R2L_df.drop(labels= ['intrusion'], axis=1)
from sklearn.preprocessing import LabelBinarizer
YR2L_train = LabelBinarizer().fit_transform(YR2L_train)

XR2L_train=np.array(XR2L_train)
YR2L_train=np.array(YR2L_train)
XR2L_train, XR2L_test, YR2L_train, YR2L_test = train_test_split(XR2L_train, YR2L_train, test_size=0.2, random_state=42)

# reshape input to be [samples, time steps, features]
XR2L_train = np.reshape(XR2L_train, ( XR2L_train.shape[0], XR2L_train.shape[1],1 ))
XR2L_test = np.reshape(XR2L_test, ( XR2L_test.shape[0],  XR2L_test.shape[1],1 ))

In [79]:
# create CNN model
model3 = Sequential()

# add convolutional layer with 32 filters, kernel size of 3x3 and activation function ReLU
model3.add(Conv1D(32, kernel_size=3,activation='relu', input_shape=( XR2L_train.shape[1],XR2L_train.shape[2])))

# add max pooling layer
model3.add(MaxPool1D(pool_size=3))

# add flatten layer to convert output of convolutional layer to 1D array
model3.add(Flatten())


# add dense layer with 128 neurons and activation function ReLU
model3.add(Dense(128, activation='relu'))

# add dropout layer with rate of 0.2 to prevent overfitting
model3.add(Dropout(0.2))

# add output layer with 1 neuron and sigmoid activation function for binary classification
model3.add(Dense(1, activation='sigmoid'))

# compile model with binary crossentropy loss function and Adam optimizer
model3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# print model summary
model3.summary()


Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_8 (Conv1D)           (None, 120, 32)           128       
                                                                 
 max_pooling1d_7 (MaxPooling  (None, 40, 32)           0         
 1D)                                                             
                                                                 
 flatten_7 (Flatten)         (None, 1280)              0         
                                                                 
 dense_14 (Dense)            (None, 128)               163968    
                                                                 
 dropout_15 (Dropout)        (None, 128)               0         
                                                                 
 dense_15 (Dense)            (None, 1)                 129       
                                                     

In [80]:
history = model3.fit(XR2L_train, YR2L_train, batch_size=32, epochs=50, validation_data=(XR2L_test, YR2L_test))

Epoch 1/50
2024/2024 [==============================] - 73s 9ms/step - loss: 0.0706 - accuracy: 0.9717 - val_loss: 0.0512 - val_accuracy: 0.9812
Epoch 2/50
2024/2024 [==============================] - 20s 10ms/step - loss: 0.0515 - accuracy: 0.9790 - val_loss: 0.0475 - val_accuracy: 0.9830
Epoch 3/50
2024/2024 [==============================] - 20s 10ms/step - loss: 0.0471 - accuracy: 0.9810 - val_loss: 0.0447 - val_accuracy: 0.9810
Epoch 4/50
2024/2024 [==============================] - 19s 9ms/step - loss: 0.0444 - accuracy: 0.9822 - val_loss: 0.0418 - val_accuracy: 0.9832
Epoch 5/50
2024/2024 [==============================] - 20s 10ms/step - loss: 0.0427 - accuracy: 0.9823 - val_loss: 0.0395 - val_accuracy: 0.9872
Epoch 6/50
2024/2024 [==============================] - 19s 10ms/step - loss: 0.0405 - accuracy: 0.9832 - val_loss: 0.0386 - val_accuracy: 0.9862
Epoch 7/50
2024/2024 [==============================] - 20s 10ms/step - loss: 0.0394 - accuracy: 0.9837 - val_loss: 0.0413 - v

# - U2R

In [82]:
U2R_df.drop(labels= ['label'], axis=1, inplace=True)
YU2R_train = U2R_df['intrusion']
XU2R_train = U2R_df.drop(labels= ['intrusion'], axis=1)
from sklearn.preprocessing import LabelBinarizer
YU2R_train = LabelBinarizer().fit_transform(YU2R_train)

XU2R_train=np.array(XU2R_train)
YU2R_train=np.array(YU2R_train)
XU2R_train, XU2R_test, YU2R_train, YU2R_test = train_test_split(XU2R_train, YU2R_train, test_size=0.2, random_state=42)

# reshape input to be [samples, time steps, features]
XU2R_train = np.reshape(XU2R_train, ( XU2R_train.shape[0], XU2R_train.shape[1],1 ))
XU2R_test = np.reshape(XU2R_test, ( XU2R_test.shape[0],  XU2R_test.shape[1],1 ))

# create CNN model
model4 = Sequential()

# add convolutional layer with 32 filters, kernel size of 3x3 and activation function ReLU
model4.add(Conv1D(32, kernel_size=3,activation='relu', input_shape=( XU2R_train.shape[1],XU2R_train.shape[2])))

# add max pooling layer
model4.add(MaxPool1D(pool_size=3))

# add flatten layer to convert output of convolutional layer to 1D array
model4.add(Flatten())


# add dense layer with 128 neurons and activation function ReLU
model4.add(Dense(128, activation='relu'))

# add dropout layer with rate of 0.2 to prevent overfitting
model4.add(Dropout(0.2))

# add output layer with 1 neuron and sigmoid activation function for binary classification
model4.add(Dense(1, activation='sigmoid'))

# compile model with binary crossentropy loss function and Adam optimizer
model4.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# print model summary
model4.summary()

history = model4.fit(XU2R_train, YU2R_train, batch_size=32, epochs=50, validation_data=(XU2R_test, YU2R_test))

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_9 (Conv1D)           (None, 120, 32)           128       
                                                                 
 max_pooling1d_8 (MaxPooling  (None, 40, 32)           0         
 1D)                                                             
                                                                 
 flatten_8 (Flatten)         (None, 1280)              0         
                                                                 
 dense_16 (Dense)            (None, 128)               163968    
                                                                 
 dropout_16 (Dropout)        (None, 128)               0         
                                                                 
 dense_17 (Dense)            (None, 1)                 129       
                                                     

1930/1930 [==============================] - 21s 11ms/step - loss: 0.0023 - accuracy: 0.9993 - val_loss: 0.0048 - val_accuracy: 0.9992
Epoch 49/50
1930/1930 [==============================] - 21s 11ms/step - loss: 0.0025 - accuracy: 0.9994 - val_loss: 0.0044 - val_accuracy: 0.9992
Epoch 50/50
1930/1930 [==============================] - 20s 10ms/step - loss: 0.0023 - accuracy: 0.9993 - val_loss: 0.0040 - val_accuracy: 0.9991
